In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
import rasterio
import geopandas as gpd
import waterdetect as wd

import time
from datetime import datetime
import calendar

import shapely.geometry
from shapely.geometry import mapping
from shapely.geometry import Point, LineString
import shapely.geometry

from skimage.measure import label, regionprops
from skimage.morphology import skeletonize
from skimage.graph import MCP_Geometric
import cv2 as cv

from itertools import combinations


from scipy import ndimage


import scipy.stats

import collections
import math

import itertools
import operator
from collections import defaultdict

from IPython.display import clear_output

### shared funcs

In [2]:
#Shared functions

def create_new_dir(outdir):
    ##Create output file
    try:
        os.mkdir(outdir)
        print(f'{outdir} created to export results')
    except:
        pass 

def export_netcdf (outdir, target_da, filename):
    #create output dir
    out_dir = os.path.join(outdir, 'netCDF')
    create_new_dir(out_dir)
    #set out file path
    out_file = os.path.join(out_dir, filename)
    #convert datarray to dataset
    DS = target_da.to_dataset(name='water')
    #add crs to attrs
    DS.water.attrs['crs'] = str(target_da.rio.crs)
    #create dict for each variable for encoding
    comp = dict(zlib=True, complevel=6)   ### need that much compression?
    encoding = {var: comp for var in DS.data_vars}
    #export compressed netCDF
    DS.to_netcdf(out_file, encoding=encoding)                     
    return out_file

def estimate_time (total_iterations, t_begin, t_end, est_time_lst):
    #calculate time to execute one iteration
    est_time_lst.append(t_end-t_begin)
    est_time_lst_av = sum(est_time_lst) / len(est_time_lst)  
    total_est_seconds = total_iterations * est_time_lst_av
    #convert total seconds to h,m,s
    m, s = divmod(total_est_seconds, 60)
    h, m = divmod(m, 60)
    if h != 0:
        if h == 1:
            print(f"Estimated remaining time: {h:0.0f} hour, {m:0.0f} minutes and {s:0.0f} seconds")
        else:
            print(f"Estimated remaining time: {h:0.0f} hours, {m:0.0f} minutes and {s:0.0f} seconds")
    else:
        if m == 1:
            print(f"Estimated remaining time: {m:0.0f} minute and {s:0.0f} seconds")
        elif m == 0:
            print(f"Estimated remaining time: {s:0.0f} seconds")
        else:
            print(f"Estimated remaining time: {m:0.0f} minutes and {s:0.0f} seconds")
    #update total_iterations
    total_iterations -= 1
    return total_iterations

### wd_batch

In [3]:
class WaterDetectBatch():

    def __init__(self, input_img, ini_file, rcor_extent, buffer = 1000, img_ext = '.tif', reg = None, max_cluster = None, export_tif = False):
        self.input_img = input_img
        self.ini_file = ini_file
        self.rcor_extent = rcor_extent
        self.buffer = buffer
        self.img_ext = img_ext
        self.reg = reg
        self.max_cluster = max_cluster
        self.export_tif = export_tif
        self.outdir = None

    def validate (self):
        def _is_valid_input_img ():
            print('Checking input data...')
            #check if netCDF
            if self.input_img.endswith('.nc') == True:
                #open netCDF file as xarray
                self.input_img = rxr.open_rasterio(self.input_img, decode_coords='all', chunks={'x': 1000, 'y': 1000})
                crs = self.input_img.rio.crs
                self.input_img = self.input_img.to_array(dim='band')
                #fill outliers and set nodata values to 0
                self.input_img = xr.where(((self.input_img > 0) & (self.input_img < 20000)), self.input_img, 0)
                #for some reason ds lost crs after 'where'
                self.input_img = self.input_img.rio.write_crs(crs)
                #fill nodata and add to attrs
                self.input_img = self.input_img.fillna(0)             
                self.input_img.attrs['_FillValue'] = 0
                # rename and transpose dims
                # self.input_img = self.input_img.rename({'variable': 'band'})
                self.input_img = self.input_img.transpose('time', 'band', 'y', 'x')
                #check if arrays have at least VNIR bands
                assert len(self.input_img.band) <= 4, 'Not enough data. Dataset must have at least 4 bands (B,G,R,NIR)'
                #check if arrays have at least VNIR bands
                if len(self.input_img.band) == 4:
                    print('Reminder: 4 bands in source must be stacked as B,G,R,NIR')
                else:
                    print(f'{len(self.input_img.data_vars)} bands found. Reminder: First 6 bands must be stacked as B,G,R,NIR,SWIR1,SWIR2')
                #check if arrays have time dimension
                assert 'time' in self.input_img.dims, 'Dataset missing "time" dimension'
                return self.input_img, len(self.input_img.band)
            #check if folder  
            elif self.input_img.endswith('.nc') == False and os.path.isdir(self.input_img) == True:
                #create list of image paths
                self.input_img = [os.path.join(self.input_img, f) for f in os.listdir(self.input_img) if f.endswith(self.img_ext)]
                #check if date is valid and if crs and pixel size are the same
                crs_lst = []
                res_lst = []
                band_lst = []
                for img in self.input_img:
                    pimg = rxr.open_rasterio(img)
                    assert pd.Timestamp(os.path.basename(img)[3:13]), f'{img} Invalid Filename. Valid example: s2_2018-01-01.tif (sensor_id (2 letters) _ date (yyyy-mm-dd))'
                    crs_lst.append(pimg.rio.crs)
                    res_lst.append(pimg.rio.resolution()[0])
                    band_lst.append(pimg.shape[0])
                assert len(set(crs_lst)) == 1, f'Check projection. Images must have same EPSG'
                assert len(set(res_lst)) == 1, f'Check spatial resolution. Images must have same pixel size'
                assert len(set(band_lst)) == 1, f'Check spatial resolution. Images must have same number of bands'
                #check if arrays have at least VNIR bands
                if band_lst[0] == 4:
                    print(f'Reminder: 4 bands in source must be stacked as B,G,R,NIR')
                else:
                    print(f'{band_lst[0]} bands found. Reminder: First 6 bands must be stacked as B,G,R,NIR,SWIR2')
                #convert raster time series to dataarray
                self.input_img = sorted(self.input_img)
                filenames_time = pd.DatetimeIndex([pd.Timestamp(os.path.basename(f)[3:13]) for f in self.input_img])
                time = xr.Variable('time', filenames_time)
                self.input_img = xr.concat([rxr.open_rasterio(f, chunks={'x': 1000, 'y': 1000}) for f in self.input_img], dim=time)
                self.input_img = self.input_img.fillna(0)             
                self.input_img.attrs['_FillValue'] = 0
                return self.input_img, band_lst[0]
            #if provided input not netCDF or folder raise error
            else:
                raise ValueError('Pass input_img as a valid directory or netCDF file (.nc)')
        #check input images or DataArray
        self.input_img, n_bands = _is_valid_input_img ()        
        #check if rcor_extent is a shapefile
        assert self.rcor_extent.endswith('.shp'), 'Pass river corridor extent (rcor_extent) as .shp'
        #check .ini extension
        assert self.ini_file.endswith('.ini'), "Use WaterDetect .ini file"
        print('Input data validated.')
        return self.input_img, n_bands

    def wd_batch(self):
        def change_ini (self, n_bands):
            #open ini file
            a_file = open(self.ini_file, "r")
            #read txt lines
            list_of_lines = a_file.readlines()
            #check number of bands and change ini file accordingly
            if n_bands == 4:
                bands = ['Blue', 'Green', 'Red', 'Nir']
                if self.max_cluster == None and self.reg == None:
                    self.max_cluster = 6
                    self.reg = 0.07
                list_of_lines[84] = "#\t\t    ['mndwi', 'ndwi', 'Mir2'],\n"
                list_of_lines[104] = "\t\t    ['ndwi', 'Nir' ],\n"
            else:
                bands=['Blue', 'Green', 'Red', 'Nir', 'Mir', 'Mir2']
                if self.max_cluster == None and self.reg == None:
                    self.max_cluster = 3
                    self.reg = 0.08
                list_of_lines[84] = "\t\t    ['mndwi', 'ndwi', 'Mir2'],\n"
                list_of_lines[104] = "#\t\t    ['ndwi', 'Nir' ],\n"
            #set max_cluster and reg
            list_of_lines[117] = 'regularization = ' + str(self.reg) + '\n'
            list_of_lines[124] = 'max_clusters = ' + str(self.max_cluster) + '\n'
            #open and edit default .ini file
            a_file = open(self.ini_file, "w")
            a_file.writelines(list_of_lines)
            a_file.close()
            return self.ini_file, bands

        def buffer_clip_aoi (self):
            #read rcor_extent as geodataframe
            gpd_rcor_extent = gpd.read_file(self.rcor_extent)
            #create a buffer using buffer value
            rcor_extent_buffer = gpd_rcor_extent.buffer(self.buffer)
            #reproject aoi if needed
            if rcor_extent_buffer.crs.to_epsg() != self.input_img.rio.crs.to_epsg():
                rcor_extent_buffer = rcor_extent_buffer.to_crs(self.input_img.rio.crs.to_epsg())
            #clip raster to aoi
            self.input_img = self.input_img.rio.clip(rcor_extent_buffer.geometry.apply(mapping), rcor_extent_buffer.crs)
            return self.input_img

        def create_wd_dict (self, img_target, bands):
            #rescale bands
            div_factor = 10000
            cube = img_target/div_factor
            arrays = {}   
            #create dict for wd input
            for i, layer in enumerate(bands):
                arrays[layer] = cube[i].values
            #create dummy DataArray
            water_xarray = img_target.isel(band=1)
            return arrays, water_xarray

        def wd_mask (self, img_target, water_xarray):
            #create_wd_dict from data arrays
            arrays, water_xarray = create_wd_dict(self, img_target, bands)
            #run wd depending on the number of bands
            if len(bands) == 4:
                # As we don't have Mir band, it will cheat the water detect (to be corrected in the WaterDetect next version)
                arrays['mndwi'] = np.zeros_like(arrays['Green'])
                arrays['mbwi'] = np.zeros_like(arrays['Green'])
                arrays['Mir2'] = np.zeros_like(arrays['Green'])
                invalid_mask = (arrays['Nir'] == 0)
                #run wd
                wmask = wd.DWImageClustering(bands=arrays, bands_keys=['ndwi', 'Nir'], invalid_mask=invalid_mask, config=config, glint_processor=None)
                mask = wmask.run_detect_water()
                water_xarray.values= wmask.water_mask
                water_xarray.rio.write_nodata(-1, inplace=True)
                clear_output()
                return water_xarray
            else:
                invalid_mask = (arrays['Mir2'] == 0)
                #run wd
                wmask = wd.DWImageClustering(bands=arrays, bands_keys=['mndwi', 'ndwi', 'Mir2'], invalid_mask=invalid_mask, config=config, glint_processor=None)
                mask = wmask.run_detect_water()
                water_xarray.values= wmask.water_mask
                water_xarray.rio.write_nodata(-1, inplace=True)
                clear_output()
                return water_xarray

        #execute class
        import time
        #create output dir
        self.outdir = os.path.join(os.path.dirname(os.path.abspath(self.ini_file)), 'results_RiverConnect')
        outdir = self.outdir
        create_new_dir(self.outdir)
        self.outdir = os.path.join(self.outdir, 'wd_batch')
        create_new_dir(self.outdir)
        #create dir to export tif files
        if self.export_tif == True:
            tif_out_dir = os.path.join(self.outdir, 'wmask_tif')
            create_new_dir(tif_out_dir)
        #validate inputs
        self.input_img, n_bands = self.validate ()
        #edit ini file
        self.ini_file, bands = change_ini (self, n_bands)
        #configure wd
        config = wd.DWConfig(config_file= self.ini_file)
        config.detect_water_cluster
        #image buffer aoi and clip
        self.input_img = buffer_clip_aoi (self)
        #create lists to store results
        wd_lst = []
        time_lst = []
        est_time_lst = []
        total_iterations = len(self.input_img)
        #iterate through timesteps
        for img_target in self.input_img:
            t_begin = time.perf_counter()
            #retrieve date
            date = pd.to_datetime(str(img_target.time.values)).strftime('%Y-%m-%d')
            print(f'Executing {date}')
            #append date to list
            time_lst.append(date)
            #execute wd
            water_xarray = wd_mask (self, img_target, bands)
            #append results
            wd_lst.append(water_xarray)
            #export rasters
            if self.export_tif == True:
                water_xarray.rio.to_raster(os.path.join(tif_out_dir, str(date) + '.tif'), compress='lzw')
            t_end = time.perf_counter()
            #estimate time
            total_iterations = estimate_time (total_iterations, t_begin, t_end, est_time_lst)
        clear_output()
        print('Working on results...')
        #create a time DataArray
        time = xr.Variable('time', pd.DatetimeIndex(time_lst))
        #concatenate results
        da_wmask = xr.concat(wd_lst, dim=time).chunk(chunks= {'x': 1000, 'y': 1000})
        #make sure results have crs
        da_wmask.rio.write_crs(self.input_img.rio.crs, inplace=True)
        #reproject results to UTM WGS84
        da_wmask = da_wmask.rio.reproject(da_wmask.rio.estimate_utm_crs(), resolution = round(self.input_img.rio.resolution()[0],2)).chunk(chunks= {'x': 1000, 'y': 1000})
        clear_output()
        #export results to netcdf
        print('Exporting results...')
        export_netcdf (self.outdir, da_wmask, 'da_wmask.nc')
        clear_output()
        print('All done!')

        return da_wmask, outdir
    

### Metrics + Patterns

In [4]:
class Metrics():
    
    def __init__(self, da_wmask, rcor_extent, export_shp = False, export_raster = True, outdir=None, sec_length= None):
        
        self.da_wmask = da_wmask
        self.rcor_extent = rcor_extent
        self.export_shp = export_shp
        self.outdir = outdir
        
        self.sec_length = sec_length
        
        self.export_raster = export_raster
        
    def validate (self):
        
        print('Checking input data...')
        
        assert type(self.da_wmask) == str or type(self.da_wmask) == xr.core.dataarray.DataArray, 'Invalid Input. Pass da_wmask as a valid netCDF file (.nc) or DataArray'
        
        if type(self.da_wmask) == str:
            assert self.da_wmask.endswith('.nc'), 'Invalid Input. Pass da_wmask as a valid netCDF file (.nc)'
            
            if os.path.basename(os.path.dirname(os.path.abspath(da_wmask))) == 'netCDF':
                self.outdir = os.path.join(os.path.dirname((os.path.dirname(os.path.dirname(os.path.abspath(self.da_wmask))))))
            else:
                self.outdir = os.path.dirname(os.path.abspath(self.da_wmask))
                
            self.da_wmask = rxr.open_rasterio(self.da_wmask, decode_coords='all')
            crs = self.da_wmask.rio.crs
            self.resolution = self.da_wmask.rio.resolution()
            
            if type(self.da_wmask) == xr.core.dataset.Dataset:
                self.da_wmask = self.da_wmask.to_array(dim='band')
                self.da_wmask  = self.da_wmask.squeeze(dim='band')
                self.da_wmask = self.da_wmask.rio.write_crs(crs)
                
            self.da_wmask.attrs['_FillValue'] = -1
            
            assert self.da_wmask.rio.crs != None, 'CRS not found.'
            assert self.da_wmask.dims == ('time', 'y', 'x'), "Dimensions not ('time', 'y', 'x')"
        
        else: 
            
            assert self.da_wmask.rio.crs != None, 'CRS not found.'
            assert self.da_wmask.dims == ('time', 'y', 'x'), "Dimensions not ('time', 'y', 'x')"
        
        #check rcor_extent extension
        assert self.rcor_extent.endswith('.shp'), 'Pass river corridor extent (rcor_extent) as .shp'
        
        clear_output()
        return self.da_wmask
    
    def fill_nodata (self, bbox_da_mask):
        #fill no data with condition
            #for each time layer check if no data exists, if so, fill data with next layer pixel value 
        print('Filling NoData...')
        
        # for num, array in enumerate(bbox_da_mask):
        #     try:
        #         bbox_da_mask[num]= xr.where(bbox_da_mask[num].isin(-1) , bbox_da_mask[num+1], bbox_da_mask[num])
        #     except:
        #         try:
        #             bbox_da_mask[num]= xr.where(bbox_da_mask[num].isin(-1) , bbox_da_mask[num-1], bbox_da_mask[num])
        #         except:    
        #             try:
        #                 bbox_da_mask[num]= xr.where(bbox_da_mask[num].isin(-1) , bbox_da_mask[num+2], bbox_da_mask[num])
        #             except:
        #                 try:
        #                     bbox_da_mask[num]= xr.where(bbox_da_mask[num].isin(-1) , bbox_da_mask[num-2], bbox_da_mask[num])
        #                 except:
        #                     pass
        
        for num, array in enumerate(bbox_da_mask):
            try:
                bbox_da_mask[num]= xr.where(bbox_da_mask[num].isin(-1) , bbox_da_mask[num+1], bbox_da_mask[num])
            except:
                bbox_da_mask[num]= xr.where(bbox_da_mask[num].isin(-1) , bbox_da_mask[num-1], bbox_da_mask[num])
            try:
                bbox_da_mask[num]= xr.where(bbox_da_mask[num].isin(-1) , bbox_da_mask[num+2], bbox_da_mask[num])
            except:
                try:
                    bbox_da_mask[num]= xr.where(bbox_da_mask[num].isin(-1) , bbox_da_mask[num-2], bbox_da_mask[num])
                except:
                    pass
        
        
        clear_output()
        return bbox_da_mask#.chunk(chunks= {'x': 1000, 'y': 1000})
    
    #clip and calculate wetted area
    def input_process (self, rcor_extent, layer):
        
        # reproject xarray to UTM wgs84
        # layer = layer.rio.reproject(layer.rio.estimate_utm_crs(), resolution = 10)#, resolution = layer.rio.resolution()[0]) ##############
        layer = layer.rio.reproject(layer.rio.estimate_utm_crs(), resolution = self.resolution[0])
        self.crs_wgs = layer.rio.crs
        #reproject aoi if needed
        if rcor_extent.crs.to_epsg() != layer.rio.crs.to_epsg():
            rcor_extent= rcor_extent.to_crs(layer.rio.crs.to_epsg())
        #clip raster to aoi
        _layer = layer.rio.clip(rcor_extent.geometry, rcor_extent.crs)
        
        #copy _layer to return clipped array
        layer_clip = _layer.values.copy()
        #change any other values than 1 to zero
        _layer= xr.where(_layer== 1, _layer, 0)
        
        #Calculate euclidean distance transform array
        euc_dist_trans_array = ndimage.distance_transform_edt(_layer)
        
        #group connected pixels into groups
        pre_label= label(_layer, connectivity=2)
        pre_label= np.where(_layer == 1, pre_label, 0)
        
        _layer = _layer.rio.write_crs(self.crs_wgs)
        _layer.attrs['_FillValue'] = 0
        
        # #get each group properties
        pre_label_shapes = list(rasterio.features.shapes((pre_label.astype('int32')), transform=_layer.rio.transform()))
        data = [(value, shapely.geometry.shape(polygon).length, shapely.geometry.shape(polygon).area) for polygon, value in pre_label_shapes]

        d = defaultdict(list)
        for key, group in itertools.groupby(sorted(data), operator.itemgetter(0)):    
            len_sum = sum([float(t[1]) for t in group])
            d[key].append(len_sum)
        for key, group in itertools.groupby(sorted(data), operator.itemgetter(0)):  
            area_sum = sum([float(t[2]) for t in group])
            d[key].append(area_sum)

        d_area = {key: val[0] for key, val in d.items()}
        u,inv = np.unique(pre_label, return_inverse = True)
        area_array = np.array([d_area[x] for x in u])[inv].reshape(pre_label.shape)
        
        #total wet area
        total_wet_area= [(sum((x[1]) for key, x in d.items() if key != 0))]
        #total perimeter
        total_wet_perimeter = [(sum((x[0]) for key, x in d.items() if key != 0))]
        # AWMSI
        AWMSI = [(sum(((0.25*x[0]/np.sqrt(x[1]))*((x[1])/total_wet_area[0])) for key, x in d.items() if key != 0))]
        
        # AWMPS
        try:
            AWMPS = np.average([x[1] for key, x in d.items() if key != 0], weights = [x[1] for key, x in d.items() if key != 0])
        except:
            AWMPS = 0       

        #skeletonize water mask
        _layer.values = skeletonize(_layer, method='lee')
        _layer= _layer.chunk(chunks= {'x': 1000, 'y': 1000})
        layer_skel = _layer.copy()
        #change any other values than 1 to zero
        _layer= xr.where(_layer== 0, _layer, 1)
        # find endpoints by checking neighbourhood
        kernel = np.uint8([[1,  1, 1],
                            [1, 10, 1],
                            [1,  1, 1]])
        #calculate kernel
        end_point_filter= cv.filter2D(_layer.values, -1, kernel)
        #retrieve 11 and 13 values - representing potential endpoints
        end_points= np.argwhere((end_point_filter==11) | (end_point_filter==13))
        #create lists to store results and check duplicates
        _dict_dist= {}
        dup_list= []
        #group connected pixels into groups
        _layer.values= label(_layer, connectivity=2)
        _layer= _layer.chunk(chunks= {'x': 1000, 'y': 1000})
        #iterate through end points
        for i in end_points:
            #check item index
            end_label= _layer[i[0], i[1]]
            #retrieve label
            end_label= end_label.values.item()
            dup_list.append(end_label)
            #check for dups
            count= dup_list.count(end_label)
            #if no dups found create new item, else append to existing item
            if count == 1:
                _dict_dist[end_label]= {1: {'id': 1, 'np': [i[0], i[1]]}}
            else:
                _dict_dist[end_label][count]= {'id': count, 'np': [i[0], i[1]]}
        _layer= xr.where(_layer!= 0, 1, -1)
        _layer.values= _layer.values.astype('int8')
        _layer= _layer.chunk(chunks= {'x': 1000, 'y': 1000})
        return layer_clip, total_wet_area, total_wet_perimeter, AWMSI, layer_skel, end_points, _layer, _dict_dist, area_array, euc_dist_trans_array, AWMPS
    
    def filter_max_dist(self, _dict_dist):
        #create lists to store results
        filtered_max_list= []
        #iterate through dicts (each label)
        for _dict in _dict_dist:
            #create list to store partial results
            max_dist_list=[]
            #calculate all possible combinations for end points
            comb_list= list(combinations(_dict_dist[_dict].keys(), 2))
            #calculate euclidean dist for each combination
            for item in comb_list:
                x= np.array(_dict_dist[_dict][item[0]]['np'])
                y= np.array(_dict_dist[_dict][item[1]]['np'])
                dist = np.linalg.norm(x - y)
                _dict_comb_filter= {'keys': item, 'dist': dist, 'start': x, 'end': y}
                max_dist_list.append(_dict_comb_filter)
            #sort calculated distance
            max_dist_list.sort(key=lambda x: x['dist'], reverse=True)
            #select greatest five distances and append to results list
            max_dist_list= max_dist_list[0:5]
            filtered_max_list.append(max_dist_list)
        #create list to store partial results    
        small_list= []
        #iterate through results list and filter values bigger than 2 pixels
        for num, group in enumerate(filtered_max_list):
            if ((sum((item['dist']) for item in group))) <= 2:
                small_list.append(num)
        filtered_max_list = [v for i, v in enumerate(filtered_max_list) if i not in small_list]
        #export results as a dict
        filtered_max_dist={}
        for num, value in enumerate(filtered_max_list):
            filtered_max_dist[num]= value
        return filtered_max_dist

    def calculate_connectivity_properties(self, _layer, filtered_max_dist):
        def least_cost(m, start, end):
            #calculate minimum cost path
            costs, traceback_array = m.find_costs([start], [end])
            return m.traceback(end), costs[end]
        def np_position_to_coord_point_list (np_position, _layer):
            #create list to store coord results
            coord_list= []
            for i in np_position:
                #retrieve x
                x= _layer['x'][i[1]]
                x= np.reshape(x.values, (1,1))
                x= x[0][0].astype('float32')
                #retrieve y
                y= _layer['y'][i[0]]
                y= np.reshape(y.values, (1,1))
                y= y[0][0].astype('float32')
                #append x,y as shapely objects
                coord_list.append(Point(x, y))
            return coord_list
        #create dict to store main results
        _dict_wet_prop={}
        #initialize skimage minimum cost path 
        m = MCP_Geometric(_layer, fully_connected=True)
        #create list to store results
        n_pools_index_lst = []
        lines_index_lst = []
        lines_index_lst_width = []
        length_area_index_lst = []
        #iterate through filtered_max_dist
        for idf in filtered_max_dist:
            #create dict to store partial results
            _dict_comb={}
            #calculate minimum cost path for each label
            for num, item in enumerate(filtered_max_dist[idf], start=1):
                cost_path, dist= least_cost(m, tuple(item['start']), tuple(item['end']))
                _dict_comb[num]= {'dist': dist, 'path': cost_path}
            try:
                #retrieve max distance for label
                max_dist= max((d['dist']) for d in _dict_comb.values())
                #retrieve results for max dist label
                for d in _dict_comb.values():
                    if d['dist']== max_dist:
                        #retrieve results for exporting xarray
                        n_pools_index_lst.append(d['path'][0])
                        n_pools_index_lst.append(d['path'][-1])
                        lines_index_lst.extend(d['path'])
                        
                        #retrieve results for exporting shp
                        coord_list= np_position_to_coord_point_list (d['path'], _layer)
                        line_path = LineString(coord_list)
                        _dict_wet_prop[idf]= {'coord_start': Point(line_path.coords[0]),
                                              'coord_end': Point(line_path.coords[-1]),
                                              'length': line_path.length, 
                                              'linestring': line_path}
                        
                        length_area_index_lst.append((d['path'][0], line_path.length))
                        lines_index_lst_width.append((d['path'], line_path.length))
                    else:
                        pass
            except:
                pass
        return _dict_wet_prop, n_pools_index_lst, lines_index_lst, length_area_index_lst, lines_index_lst_width
    
    def save_results_xr (self, layer_clip, index_lst):
        #create dummy array with nodata and zeros
        dummy = layer_clip#.load()
        dummy.fillna(-1)
        dummy = xr.where(dummy == -1, dummy, 0)
        #retrieve index from valid points
        ind_x = []
        ind_y = []
        for indx in index_lst:
            ind_x.append(indx[0])
            ind_y.append(indx[1])
        #create xarray for idx and idy
        dind_x = xr.DataArray(ind_x, dims=['points'])
        dind_y = xr.DataArray(ind_y, dims=['points'])
        try:
            #populate valid indices with 1
            dummy[dind_x, dind_y] = 1
        except:
            print('ERROR: Check Layer')
        return dummy

    def save_shp (self, _dict_wet_prop):
        
        out_shp_dir = os.path.join(self.outdir, 'shp')
        create_new_dir (out_shp_dir)
        
        n_pools_gdf = gpd.GeoDataFrame()
        d_line_gdf = gpd.GeoDataFrame()
        
        for d in _dict_wet_prop:
            start = {'date': self.date, 'length': _dict_wet_prop[d]['length'],  'geometry': _dict_wet_prop[d]['coord_start']}
            n_pools_gdf= n_pools_gdf.append(start, ignore_index=True)
            end = {'date': self.date, 'length': _dict_wet_prop[d]['length'], 'geometry': _dict_wet_prop[d]['coord_end']}
            n_pools_gdf= n_pools_gdf.append(end, ignore_index=True)

            line = {'date': self.date, 'length': _dict_wet_prop[d]['length'], 'geometry': _dict_wet_prop[d]['linestring']}
            d_line_gdf= d_line_gdf.append(line, ignore_index=True)

        n_pools_gdf= n_pools_gdf.set_crs(self.crs_wgs)
        n_pools_gdf.to_file(os.path.join(out_shp_dir, 'dpoints_' + self.date + '.shp'))

        d_line_gdf= d_line_gdf.set_crs(self.crs_wgs)
        d_line_gdf.to_file(os.path.join(out_shp_dir, 'rline_' + self.date + '.shp'))  
        
    
    #### CALC METRICS #########

    # def calc_metrics (self):
    def ini_metrics (self):

        pdm = self.pd_metrics[['wet_area_km2', 'wet_perimeter_km2', 'wet_length_km', 'sec_area', 'npools', 'AWMSI', 'AWRe', 
                               'AWMPS', 'AWMPL', 'AWMPW', 'MPW']].copy()

        pdm['sec_length'] = self.sec_length

        pdm.loc[:,'APSEC'] = ((pdm['wet_area_km2']/pdm['sec_area'])*100).replace(np.inf, 0.0)
        pdm.loc[:,'LPSEC'] = ((pdm['wet_length_km']/self.sec_length)*100).replace(np.inf, 0.0)

        pdm.loc[:,'PDS'] = (pdm['npools']/pdm['wet_area_km2']).replace(np.inf, 0.0)
        pdm.loc[:,'PDT'] = (pdm['npools']/pdm['wet_length_km']).replace(np.inf, 0.0)       

        pdm.loc[:, :] = pdm.replace(np.nan, 0)
        pdm.loc[:, :] = pdm.replace(np.inf, 0)           

        pdm['date'] = pd.to_datetime(self.pd_metrics['date'], format='%Y-%m-%d')

        col_order = ['date', 'sec_area', 'sec_length', 'wet_area_km2', 'wet_perimeter_km2', 'wet_length_km', 'npools', 
                      'AWMSI', 'AWRe', 'AWMPS', 'AWMPL', 'AWMPW', 'MPW', 'APSEC', 'LPSEC', 'PDS', 'PDT']

        pdm = pdm[col_order]

        return pdm 
    
    
    def pixel_level (self, da_area, pdm):

        def export_raster(self, data, crs, fill, filename):
            #set dataarray nodata
            data.attrs['_FillValue'] = fill
            #set crs
            data = data.rio.write_crs(crs)
            #export raster
            data.rio.to_raster(os.path.join(self.outdir_raster, filename), compress='lzw', lock = True)

        def persist_area (self, da_area, pdm):
            
            # if type(self.da_area) == str:
            #     # self.da_area = rxr.open_rasterio(self.da_area, decode_coords='all')
            #     self.da_area = xr.open_dataset(self.da_area, decode_coords='all')               
            # #save crs
            
            self.crs = da_area.rio.crs
            
            #count timesteps
            total_obs = int(da_area['time'].count().values)
            #calculate pixel persistence
            p_area = (da_area.sum(dim='time')/total_obs).astype('float32')
            #change values less than zero to nodata
            p_area = xr.where(p_area >= 0, p_area, -1)
            #set crs
            p_area = p_area.rio.write_crs(self.crs)
            
            pdm['PP'] = np.nanmean(xr.where(p_area >= 0.25, p_area, np.nan).values)
            pdm['RA'] = (((np.nansum(xr.where(p_area >= 0.9, 1, np.nan).values))*self.resolution[0]*self.resolution[0])/10**6)
            #Group 1 only >
            pdm['PP_m25'] = (((np.nansum(xr.where(p_area >= 0.25, 1, np.nan).values))*self.resolution[0]*self.resolution[0])/10**6)
            pdm['PP_m45'] = (((np.nansum(xr.where(p_area >= 0.45, 1, np.nan).values))*self.resolution[0]*self.resolution[0])/10**6)
            pdm['PP_m50'] = (((np.nansum(xr.where(p_area >= 0.5, 1, np.nan).values))*self.resolution[0]*self.resolution[0])/10**6)
            pdm['PP_m65'] = (((np.nansum(xr.where(p_area >= 0.65, 1, np.nan).values))*self.resolution[0]*self.resolution[0])/10**6)
            pdm['PP_m75'] = (((np.nansum(xr.where(p_area >= 0.75, 1, np.nan).values))*self.resolution[0]*self.resolution[0])/10**6)
            
            #Group 2 only intervals            
            pdm['PP_i25_50'] = (((np.nansum(xr.where((p_area >= 0.25) & (p_area <= 0.5), 1, np.nan).values))*self.resolution[0]*self.resolution[0])/10**6)
            pdm['PP_i50_75'] = (((np.nansum(xr.where((p_area >= 0.5) & (p_area <= 0.75), 1, np.nan).values))*self.resolution[0]*self.resolution[0])/10**6)
            pdm['PP_i75_90'] = (((np.nansum(xr.where((p_area >= 0.75) & (p_area <= 0.9), 1, np.nan).values))*self.resolution[0]*self.resolution[0])/10**6)
            
            #Group 3 only intervals            
            pdm['PP_i25_45'] = (((np.nansum(xr.where((p_area >= 0.25) & (p_area <= 0.45), 1, np.nan).values))*self.resolution[0]*self.resolution[0])/10**6)
            pdm['PP_i45_65'] = (((np.nansum(xr.where((p_area >= 0.45) & (p_area <= 0.65), 1, np.nan).values))*self.resolution[0]*self.resolution[0])/10**6)
            pdm['PP_i65_90'] = (((np.nansum(xr.where((p_area >= 0.65) & (p_area <= 0.9), 1, np.nan).values))*self.resolution[0]*self.resolution[0])/10**6)

            # pdm.loc['PP'] = np.nanmean(xr.where(p_area >= 0.25, p_area, np.nan).water.values)
            # pdm.loc['P90_km2'] = (((np.nansum(xr.where(p_area >= 0.9, 1, np.nan).water.values))*100)/10**6)
            
            if self.export_raster == True:           
                # export water persistence
                export_raster(self, p_area, self.crs, -1, 'pwater_area.tif')

            return pdm
        
        return persist_area (self, da_area, pdm)
        
    
    def run(self):
        import time
        
        def create_dataarray (_layer, layer_metric, date_list):
            da = xr.DataArray(data = layer_metric,
                              dims= ['time', 'y', 'x'],
                              coords=dict(
                                            time = date_list,
                                            y=_layer.coords['y'],
                                            x=_layer.coords['x'],
                                )
                            )
            da.rio.write_crs(self.crs_wgs, inplace=True)
            da.attrs['crs'] = str(self.crs_wgs)
            da.attrs['_FillValue'] = -1
            return da
        

            
        #execute metrics        
        self.da_wmask = self.validate()
        
        self.outdir = os.path.join(self.outdir, 'metrics')
        first_path = self.outdir
        create_new_dir(self.outdir)
        print('Results from Metrics module will be exported to ', self.outdir)
        
        #read rcor_extent .shp as a gdf
        rcor_extent = gpd.read_file(self.rcor_extent)
        for polygon in rcor_extent.iloc:
            
            da_area = 0
            da_npools = 0
            da_rlines = 0
            
            seg_area = polygon.geometry.area/10**6
            
            aoi_poly = gpd.GeoDataFrame({'geometry': [polygon.geometry]}, crs=rcor_extent.crs)
            bbox = aoi_poly.to_crs(self.da_wmask.rio.crs.to_epsg())
            bbox_da_mask = self.da_wmask.rio.clip_box(bbox.bounds.iloc[0][0], bbox.bounds.iloc[0][1], 
                                                      bbox.bounds.iloc[0][2], bbox.bounds.iloc[0][3], auto_expand=True)
            
            bbox_da_mask = self.fill_nodata(bbox_da_mask)
            
            #create lists to store results
            date_list= []
            length_list= []
            n_pools_list= []
            total_wet_area_list= []
            total_wet_perimeter_list = []
            
            AWMSI_arr = np.empty([0])
            AWRe_arr = np.empty([0])
            AWMPS_arr = np.empty([0])
            AWMPL_arr = np.empty([0])
            AWMPW_arr = np.empty([0])
            
            MPW_arr = np.empty([0])
            
            area_np_lst = []
            length_np_lst = []
            d_point_np_lst = []
            
            if len(rcor_extent.index) > 1:
                self.outdir = os.path.join(first_path, str(polygon.name))
                create_new_dir(self.outdir)
                print('Results from Metrics module will be exported to ', self.outdir)
            
            est_time_lst = []
            total_iterations = len(self.da_wmask)
            #execute metrics

            for num, layer in enumerate(bbox_da_mask): #(self.da_wmask):

                print('Calculating metrics...')
                self.date = pd.to_datetime(str(self.da_wmask.time.values[num])).strftime('%Y-%m-%d')
                print(f'Executing {self.date}')

                #time
                t_begin = time.perf_counter()
                
                layer_clip, total_wet_area, total_wet_perimeter, AWMSI, layer_skel,\
                end_points, _layer, _dict_dist, area_array, euc_dist_trans_array, AWMPS = self.input_process (aoi_poly, layer)
                
                print('input_process ready')
                filtered_max_dist = self.filter_max_dist(_dict_dist)
                print('filtered_max_dist ready...')
                _dict_wet_prop, n_pools_index_lst, lines_index_lst, length_area_index_lst, lines_index_lst_width = self.calculate_connectivity_properties(_layer, filtered_max_dist)
                print('calculate_connectivity_properties ready...')
                
                #Calculate Elongation for each pool
                idx, idy = [x[0][0] for x in length_area_index_lst],  [y[0][1] for y in length_area_index_lst]
                length = [l[1] for l in length_area_index_lst]            
                Re_param = list(zip(area_array[idx, idy], length))
                AWRe = [np.divide( (sum([((2* (np.sqrt(x[0])/np.pi)/x[1])*(x[0])) for x in Re_param])), sum([x[0] for x in Re_param]) ) ]     
                
                width_list = []
                for idxw, length in lines_index_lst_width:
                    idx, idy = [x[0] for x in idxw],  [x[1] for x in idxw]
                    width_seg = np.mean(euc_dist_trans_array[idx, idy])
                    width_list.append((width_seg*2, length))
                                
                try:
                    AWMPW = np.average([w for w,l in width_list], weights = [a for a,l in Re_param])
                except:
                    AWMPW = 0
        
                MPW = np.average([w for w,l in width_list])

                try:                   
                    AWMPL = np.average([l for w,l in width_list], weights = [a for a,l in Re_param])
                except:
                    AWMPL = 0

                #Append results
                date_list.append((pd.to_datetime(str(bbox_da_mask[num].time.values)).strftime('%Y-%m-%d')))
                length_list.append((sum((_dict_wet_prop[d]['length']) for d in _dict_wet_prop)))
                total_wet_area_list.append(total_wet_area[0])
                total_wet_perimeter_list.append(total_wet_perimeter[0])
                n_pools_list.append(len(_dict_wet_prop))
                
                AWMSI_arr = np.concatenate((AWMSI_arr, AWMSI), axis=None)
                AWRe_arr = np.concatenate((AWRe_arr, AWRe), axis=None)
                AWMPS_arr = np.concatenate((AWMPS_arr, AWMPS), axis=None)
                AWMPL_arr = np.concatenate((AWMPL_arr, AWMPL), axis=None)
                AWMPW_arr = np.concatenate((AWMPW_arr, AWMPW), axis=None)
                
                MPW_arr = np.concatenate((MPW_arr, MPW), axis=None)
            
                try:
                    da_area = np.concatenate(([da_area, layer_clip[np.newaxis, :, :]]), axis=0)
                except:
                    da_area = layer_clip[np.newaxis, :, :]
                           
                layer_clip = np.where(layer_clip == -1, layer_clip, 0)
                # layer_clip = layer_clip[np.newaxis, :, :]
                
                def list_index (layer_clip, index_lst):
                    #retrieve index from valid points
                    ind_x = []
                    ind_y = []
                    for indx in index_lst:
                        ind_x.append(indx[0])
                        ind_y.append(indx[1])
                    layer_clip[ind_x, ind_y] = 1
                    layer_zeros = layer_clip[np.newaxis, :, :]
                    return layer_zeros
                
                layer_npools = list_index (layer_clip, n_pools_index_lst)
                try:
                    da_npools = np.concatenate(([da_npools, layer_npools]), axis=0)
                except:
                    da_npools = layer_npools

                layer_rlines = list_index (layer_clip, lines_index_lst)
                try:
                    da_rlines = np.concatenate(([da_rlines, layer_rlines]), axis=0)
                except:
                    da_rlines = layer_rlines                
                
                if self.export_shp == True:
                    try:
                        self.save_shp (_dict_wet_prop)
                    except:
                        pass

                t_end = time.perf_counter()
                clear_output()
                #estimate time
                total_iterations = estimate_time (total_iterations, t_begin, t_end, est_time_lst)
                
            clear_output()

            print('Exporting results...')           
            
            pand_dict= {'date': date_list, 'sec_area': seg_area, 'wet_area_km2': [d/10**6 for d in total_wet_area_list], 
                        'wet_perimeter_km2': [d/10**6 for d in total_wet_perimeter_list], 'wet_length_km': [d/1000 for d in length_list], 
                        'npools': n_pools_list,  'AWMSI':AWMSI_arr.tolist(), 'AWRe': AWRe_arr.tolist(), 
                        'AWMPS': [d/10**6 for d in AWMPS_arr.tolist()], 'AWMPL': [d/1000 for d in AWMPL_arr.tolist()], 
                        'AWMPW': [d*10/1000 for d in AWMPW_arr.tolist()], 'MPW': [d*10/1000 for d in MPW_arr.tolist()]}        
            
            self.pd_metrics = pd.DataFrame(data=pand_dict)
            
            da_area = create_dataarray (_layer, da_area, date_list)
            export_netcdf (self.outdir, da_area, 'da_area.nc')
            
            da_npools = create_dataarray (_layer, da_npools, date_list)
            export_netcdf (self.outdir, da_npools, 'da_npools.nc')

            da_rlines = create_dataarray (_layer, da_rlines, date_list)
            export_netcdf (self.outdir, da_rlines, 'da_rlines.nc')
            
            
            create_new_dir(self.outdir)
            create_new_dir(os.path.join(self.outdir, 'raster'))
            self.outdir_raster = os.path.join(self.outdir, 'raster')
            
            pdm = self.ini_metrics ()
            pdm = self.pixel_level (da_area, pdm)
            pdm.to_csv(os.path.join(self.outdir, 'pd_metrics.csv'))
           
            clear_output()
            
        result = pd.DataFrame()
        for i in range(0, len(rcor_extent)):
            outdir = os.path.join(first_path, f'{i}', 'pd_metrics.csv')
            df = pd.read_csv(outdir, index_col=0)
            df['Section'] = i + 1
            result = pd.concat([result, df], axis=0, ignore_index=True)

        result.to_csv(os.path.join(first_path, 'merged_segs.csv'))
        
        print('All done!')

### Main

In [5]:
class RiverConnect ():
    
    def __init__(self):
        
        pass
    
    def water_detect_batch(self, input_img, ini_file, rcor_extent, buffer = 1000, img_ext = '.tif', reg = None, max_cluster = None, export_tif = False):
        
        da_wmask, outdir = WaterDetectBatch (input_img, ini_file, rcor_extent, buffer, img_ext, reg, max_cluster, export_tif).wd_batch()

    def metrics(self, da_wmask, rcor_extent, export_shp = False, export_raster = True, outdir=None, sec_length= None):
        
        # da_area, da_npools, da_rlines, pd_metrics = Metrics (da_wmask, rcor_extent, export_shp, outdir).run()
        
        Metrics (da_wmask, rcor_extent, export_shp, export_raster, outdir, sec_length).run()     
        
        
    
#     def patterns(self, da_area=None, da_npools=None, pd_metrics=None, export_raster = False, outdir=None, sec_length= None):

#         pdm = Patterns(da_area, da_npools, pd_metrics, export_raster, outdir, sec_length).run()
         
            
    def run(self, input_img, ini_file, rcor_extent, buffer = 1000, img_ext = '.tif', reg = None, max_cluster = None, export_tif = False,
            export_shp = False, export_raster = False, outdir=None, sec_length= None):
        
        da_wmask, outdir = WaterDetectBatch (input_img, ini_file, rcor_extent, buffer, img_ext, reg, max_cluster, export_tif).wd_batch()
        
        da_area, da_npools, da_rlines, pd_metrics = Metrics (da_wmask, rcor_extent, export_shp, outdir).run()
               
        pdm = Patterns(da_area, da_npools, pd_metrics, export_raster, outdir, sec_length).run()
        
        
        # pass

In [6]:
x = RiverConnect()

In [ ]:
rcor_extent = r'D:\Chapter_2_2\Metrics_visual_analysis\shp\sva_buff_sort.shp'
da_wmask = r'D:\Chapter_2_2\Metrics_visual_analysis\unseg6_del\results_RiverConnect\wd_batch\netCDF\s2_wd_merge.nc'
sec_len = 10.1743

x.metrics(da_wmask, rcor_extent, sec_length=sec_len, export_shp = True)

In [7]:
%%time
#Full Catchment - 4 parts


### CURRENT CHAP 5

path = r'D:\Chapter_4\segments'

# path_list = [
#              # ('seg5','fr_main_channel_5.shp', 67.828), ('seg15','fr_main_channel_15.shp', 22.609), ('seg25','fr_main_channel_25.shp', 13.566), 
#              # ('seg50','fr_main_channel_50.shp', 6.783), ('seg100','fr_main_channel_100.shp', 3.391), ('seg200','fr_main_channel_200.shp', 1.696), 
#              # ('seg300','fr_main_channel_300.shp', 1.13), 
#              # ('seg400','fr_main_channel_400.shp', 0.848), ('seg500','fr_main_channel_500.shp', 0.678), 
#              ('seg600','fr_main_channel_600.shp', 0.565), ('seg700','fr_main_channel_700.shp', 0.484)]


# path_list = [('seg500','fr_main_channel_500.shp', 0.678), 
#              ('seg600','fr_main_channel_600.shp', 0.565), ('seg700','fr_main_channel_700.shp', 0.484)]

path_list = [#('seg1','sampled_pools_polygons.shp', 0.484),
             # ('seg2_edited','sampled_pools_polygons2.shp', 0.484),
             ('seg3_edited','sampled_pools_polygons3.shp', 0.484)]

nc_path = r'results_RiverConnect\wd_batch\netCDF\s2_wd_merge.nc'

for seg, shp, slen in path_list:
    da_wmask = os.path.join(path, seg, nc_path)
    rcor_extent = os.path.join(path, 'shp', shp)
    sec_len = slen
    x.metrics(da_wmask, rcor_extent, sec_length=sec_len)

All done!
Wall time: 3min 37s
